# Detección de rostros y reconocimiento facial

## Librerías

Este proyecto está desarrollado bajo la **versión 3.8 de python** por medio del entorno de **Anaconda para Windows**, además se hará uso de la última versión de **openCV** en conjunto con el algoritmo **LPBH** para la creación de un modelo para el reconocimiento facial.

> **Anaconda:** https://docs.anaconda.com/anaconda/install/windows/

> **OpenCV:** conda install -c conda-forge opencv

> **Modelo Detección de rostros:** Descargar archivo "haarcascade_frontalface_alt.xml" desde https://github.com/opencv/opencv/tree/master/data/haarcascades

In [169]:
import numpy as np 
import cv2
import os
from PIL import Image # For face recognition we will the the LBPH Face Recognizer 

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

## Captura de imágenes para entrenamiento
 
En el siguiente código la variable **id** representa el identificador por cada persona con la que se desea entrenar el modelo, este debe incrementar por cada nuevo rostro. 

Los pasos siguientes se enciende la cámara del dispositivo para obtener los rostros con ayuda del modelo para la detección de los mismo, que se encuentra en la variable: **face_cascade**, para mejorar la precisión la imagen se convierte a escala de grises y se configuran los parámetros para la detección de rostros en **1.1**, este factor controla el reescalado de la imagen que es de gran importancia para detectar rostros según su tamaño en la imagen.

En la carpeta **img_train** se almacenan los rostros obtenidos que en este ejemplo se toman un total de 20, el nombre de cada archivo contiene el formato **User.1.1** el primer valor es el de la variable **id** y el siguiente un incremental por las 20 imágenes que se van a generar.

In [170]:
def getImages(id, source):

    cap = cv2.VideoCapture(source)

    cont=0; #Contador por cada rostro detectado y almacenado

    facesLimit=30; #Cantidad de rostros a obtener

    while 1:

        ret, img = cap.read() #Inicia cámara y obtiene fotogramas
        
        if (ret):

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Convierte la imagen a escala de grises

            faces = face_cascade.detectMultiScale(gray, 1.2, 5)

            for (x,y,w,h) in faces:

                cont=cont+1;

                cv2.imwrite("img_train/User."+str(id)+ "." +str(cont)+ ".jpg", gray[y:y+h, x:x+w])

                cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

                cv2.waitKey(100)

            cv2.imshow('img',img)

            cv2.waitKey(1)

            if cont > facesLimit:

                break
        else:
            break

    cap.release()

    cv2.destroyAllWindows()

Para obtener imágenes desde la cámara se debe especificar el valor **id** (primer parámetro) y el valor **source** (segundo parámetro), para el cual especificar **0** corresponde a usar la cámara mientras que si se especifica una ruta, se tomara las capturas desde un video.

In [171]:
#getImages(1, 0) #Obtener imagenes desde camara
getImages(1, 'video_train/train_1.mp4') #Obtener imagenes desde video id:1
getImages(2, 'video_train/train_2.mp4') #Obtener imagenes desde video id:2

 ## Creación de modelo
 
El siguiente bloque contiene el entrenamiento del modelo para el reconocimiento facial, se toman las imágenes generadas en la carpeta **img_train** por cada persona y obtiene las características encontradas relacionándolas al **id** de cada imagen.

In [172]:
recognizer = cv2.face.LBPHFaceRecognizer_create()

path="img_train"

def getImagesWithID(path):

    imagePaths = [os.path.join(path, f) for f in os.listdir(path)]   # Obtiene imagenes del directorio

    faces = []

    IDs = []

    for imagePath in imagePaths: # Loop por cada imagen encontrada

        facesImg = Image.open(imagePath).convert('L') # Convierte la imagen a escala de grises

        faceNP = np.array(facesImg, 'uint8') # Genera matriz con pixeles de la imagen 

        ID= int(os.path.split(imagePath)[-1].split(".")[1]) #Obtiene id asignnado a la imagen

        faces.append(faceNP) # Asocia características de los rostros

        IDs.append(ID) # Almacena id por conjunto de imagenes analizadas

        cv2.imshow("Rostros para entrenamiento",faceNP) # Muestra en pantalla el proceso

        cv2.waitKey(10)

    return np.array(IDs), faces # Devuelve matrices por cada imagen asociada a una misma persona y el respectivo id

Ids,faces  = getImagesWithID(path) # Llama a función para obtener matrices de las imagenes

recognizer.train(faces,Ids) # Entrena el modelo

recognizer.save("face_model/trainingdata.yml") # Se almacena el modelo

cv2.destroyAllWindows() # Se cierran las ventanas abiertas

## Prueba modelo en tiempo real con la cámara del dispositivo

In [173]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
cap = cv2.VideoCapture(0)
rec = cv2.face.LBPHFaceRecognizer_create();
rec.read("face_model/trainingdata.yml")
id=0

fontface = cv2.FONT_HERSHEY_SIMPLEX
fontscale = 1
fontcolor = (255, 255, 255)

while 1:
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.5, 3)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        id,conf=rec.predict(gray[y:y+h,x:x+w])
        if(id==1):
            id="Keanu Reeves"
        if id==2:
            id="Laurence Fishburne"
        cv2.putText(img, str(id), (x,y+h), fontface, fontscale, fontcolor) 
        
    cv2.imshow('img',img)
    
    #Salir con 'ESC':
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        cv2.destroyAllWindows()
        break
        
cap.release()

cv2.destroyAllWindows()

## Prueba modelo con una imagen

In [174]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
rec = cv2.face.LBPHFaceRecognizer_create();
rec.read("face_model/trainingdata.yml")
id=0

fontface = cv2.FONT_HERSHEY_SIMPLEX
fontscale = 1
fontcolor = (255, 255, 255)

img = cv2.imread('img_test/1.jpg')

scale_percent = 100 # percent of original size

width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)

dim = (width, height)
# resize image
img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_cascade.detectMultiScale(gray, 1.5, 5)

for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    id,conf=rec.predict(gray[y:y+h,x:x+w])
    if id==1:
        id="Keanu Reeves"
    if id==2:
        id="Laurence Fishburne"
    cv2.putText(img, str(id), (x,y+h), fontface, fontscale, fontcolor) 
    
cv2.imshow('img',img)
print("\nMostrando resultado. Pulsa cualquier tecla para salir.\n")
cv2.waitKey(0)
cv2.destroyAllWindows()


Mostrando resultado. Pulsa cualquier tecla para salir.



## Prueba modelo con un video

In [175]:

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture('video_test/1.mp4')
rec = cv2.face.LBPHFaceRecognizer_create();
rec.read("face_model/trainingdata.yml")
id=0

fontface = cv2.FONT_HERSHEY_SIMPLEX
fontscale = 1
fontcolor = (255, 255, 255)

while cap.isOpened():
    ret, img = cap.read()
    if ret == True:
        scale_percent = 50 # percent of original size

        width = int(img.shape[1] * scale_percent / 100)
        height = int(img.shape[0] * scale_percent / 100)

        dim = (width, height)
        # resize image
        img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.5, 5)
        for (x,y,w,h) in faces:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
            id,conf=rec.predict(gray[y:y+h,x:x+w])
            if id==1:
                id="Keanu Reeves "
            if id==2:
                id="Laurence Fishburne "
            cv2.putText(img, str(id), (x,y+h), fontface, fontscale, fontcolor) 

        cv2.imshow('img',img)

        #Salir con 'ESC':
        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            cv2.destroyAllWindows()
            break
    else:
        break
        
cap.release()

cv2.destroyAllWindows()

## Conclusiones

>El algoritmo para reconocimiento facial forma parte de las librerías de OpenCV, este tiene sus limitantes ya que no permite obtener el nivel de precisión cuando se evalúa un rostro, por lo cual existen imprecisiones al encontrar rostros similares o desconocidos.

>Obtener imagen variadas y en distintas situaciones puede mejorar la precisión de cualquier modelo.

>El factor de reescalado en el modelo para detección de rostros puede afectar el resultado, un nivel mas bajo permite encontrar mas rostros, sin embargo, esto puede ser contra producente ya que puede llegar a tomar objetos que no representan un rostro, pero si características que el modelo considera similares a las de un rostro.
